## Trying out the Titanic Prediction Using Random Tree Forests

In [6]:
import numpy as np
import pandas as pd
from sklearn import tree

df_train = pd.read_csv("train.csv")

In [7]:
df_train.head()

PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S

Checking for which ones have the most correlation, and we'll use it to predict survival probability

In [8]:
correlation = df_train.corr()
correlation['Survived'].sort_values(ascending = False)

Survived       1.000000
Fare           0.257307
Parch          0.081629
PassengerId   -0.005007
SibSp         -0.035322
Age           -0.077221
Pclass        -0.338481
Name: Survived, dtype: float64

In [12]:
s = {'male' : 1, 'female' : 0}
df_train['Sex'] = df_train['Sex'].map(s)

In [14]:
df_train.head()

PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name  Sex   Age  SibSp  Parch  \
0                            Braund, Mr. Owen Harris    1  22.0      1      0   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...    0  38.0      1      0   
2                             Heikkinen, Miss. Laina    0  26.0      0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)    0  35.0      1      0   
4                           Allen, Mr. William Henry    1  35.0      0      0   

             Ticket     Fare Cabin Embarked  
0         A/5 21171   7.2500   NaN        S  
1          PC 17599  71.2833   C85        C  
2  STON/O2. 3101282   7.9250   NaN        S  
3            113803  53.1000  C123        S  
4            373450   8.0500   NaN        S

Add which features we would want to use to predict the outcome

In [16]:
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare']
features

['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare']

In [28]:
X = df_train[features]
Y = df_train['Survived']

In [30]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 6 columns):
Pclass    891 non-null int64
Sex       891 non-null int64
Age       714 non-null float64
SibSp     891 non-null int64
Parch     891 non-null int64
Fare      891 non-null float64
dtypes: float64(2), int64(4)
memory usage: 41.8 KB


Fill in the empty data with the median and using the imputer function

In [31]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy = 'median')
imputer.fit(X)
imputer.statistics_

array([ 3.    ,  1.    , 28.    ,  0.    ,  0.    , 14.4542])

In [33]:
X_tr = imputer.transform(X)
X_tr

array([[ 3.    ,  1.    , 22.    ,  1.    ,  0.    ,  7.25  ],
       [ 1.    ,  0.    , 38.    ,  1.    ,  0.    , 71.2833],
       [ 3.    ,  0.    , 26.    ,  0.    ,  0.    ,  7.925 ],
       ...,
       [ 3.    ,  0.    , 28.    ,  1.    ,  2.    , 23.45  ],
       [ 1.    ,  1.    , 26.    ,  0.    ,  0.    , 30.    ],
       [ 3.    ,  1.    , 32.    ,  0.    ,  0.    ,  7.75  ]])

In [35]:
X = pd.DataFrame(X_tr, columns=X.columns)
X

Pclass  Sex   Age  SibSp  Parch      Fare
0       3.0  1.0  22.0    1.0    0.0    7.2500
1       1.0  0.0  38.0    1.0    0.0   71.2833
2       3.0  0.0  26.0    0.0    0.0    7.9250
3       1.0  0.0  35.0    1.0    0.0   53.1000
4       3.0  1.0  35.0    0.0    0.0    8.0500
5       3.0  1.0  28.0    0.0    0.0    8.4583
6       1.0  1.0  54.0    0.0    0.0   51.8625
7       3.0  1.0   2.0    3.0    1.0   21.0750
8       3.0  0.0  27.0    0.0    2.0   11.1333
9       2.0  0.0  14.0    1.0    0.0   30.0708
10      3.0  0.0   4.0    1.0    1.0   16.7000
11      1.0  0.0  58.0    0.0    0.0   26.5500
12      3.0  1.0  20.0    0.0    0.0    8.0500
13      3.0  1.0  39.0    1.0    5.0   31.2750
14      3.0  0.0  14.0    0.0    0.0    7.8542
15      2.0  0.0  55.0    0.0    0.0   16.0000
16      3.0  1.0   2.0    4.0    1.0   29.1250
17      2.0  1.0  28.0    0.0    0.0   13.0000
18      3.0  0.0  31.0    1.0    0.0   18.0000
19      3.0  0.0  28.0    0.0    0.0    7.2250
20      2.0  1.0  35.0    0.0    0.0   26.0000
21      2.0  1.0  34.0    0.0    0.0   13.0000
22      3.0  0.0  15.0    0.0    0.0    8.0292
23      1.0  1.0  28.0    0.0    0.0   35.5000
24      3.0  0.0   8.0    3.0    1.0   21.0750
25      3.0  0.0  38.0    1.0    5.0   31.3875
26      3.0  1.0  28.0    0.0    0.0    7.2250
27      1.0  1.0  19.0    3.0    2.0  263.0000
28      3.0  0.0  28.0    0.0    0.0    7.8792
29      3.0  1.0  28.0    0.0    0.0    7.8958
..      ...  ...   ...    ...    ...       ...
861     2.0  1.0  21.0    1.0    0.0   11.5000
862     1.0  0.0  48.0    0.0    0.0   25.9292
863     3.0  0.0  28.0    8.0    2.0   69.5500
864     2.0  1.0  24.0    0.0    0.0   13.0000
865     2.0  0.0  42.0    0.0    0.0   13.0000
866     2.0  0.0  27.0    1.0    0.0   13.8583
867     1.0  1.0  31.0    0.0    0.0   50.4958
868     3.0  1.0  28.0    0.0    0.0    9.5000
869     3.0  1.0   4.0    1.0    1.0   11.1333
870     3.0  1.0  26.0    0.0    0.0    7.8958
871     1.0  0.0  47.0    1.0    1.0   52.5542
872     1.0  1.0  33.0    0.0    0.0    5.0000
873     3.0  1.0  47.0    0.0    0.0    9.0000
874     2.0  0.0  28.0    1.0    0.0   24.0000
875     3.0  0.0  15.0    0.0    0.0    7.2250
876     3.0  1.0  20.0    0.0    0.0    9.8458
877     3.0  1.0  19.0    0.0    0.0    7.8958
878     3.0  1.0  28.0    0.0    0.0    7.8958
879     1.0  0.0  56.0    0.0    1.0   83.1583
880     2.0  0.0  25.0    0.0    1.0   26.0000
881     3.0  1.0  33.0    0.0    0.0    7.8958
882     3.0  0.0  22.0    0.0    0.0   10.5167
883     2.0  1.0  28.0    0.0    0.0   10.5000
884     3.0  1.0  25.0    0.0    0.0    7.0500
885     3.0  0.0  39.0    0.0    5.0   29.1250
886     2.0  1.0  27.0    0.0    0.0   13.0000
887     1.0  0.0  19.0    0.0    0.0   30.0000
888     3.0  0.0  28.0    1.0    2.0   23.4500
889     1.0  1.0  26.0    0.0    0.0   30.0000
890     3.0  1.0  32.0    0.0    0.0    7.7500

[891 rows x 6 columns]

In [36]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 6 columns):
Pclass    891 non-null float64
Sex       891 non-null float64
Age       891 non-null float64
SibSp     891 non-null float64
Parch     891 non-null float64
Fare      891 non-null float64
dtypes: float64(6)
memory usage: 41.8 KB


Try to scale everything so that they'll all be in the same range

In [38]:
from sklearn.preprocessing import MinMaxScaler
age_scaler = MinMaxScaler()
fare_scaler = MinMaxScaler()

In [39]:
X['Age'] = age_scaler.fit_transform(X['Age'].values.reshape(len(X),1),1)
X['Fare'] = fare_scaler.fit_transform(X['Fare'].values.reshape(len(X),1),1)

In [40]:
X.head()

Pclass  Sex       Age  SibSp  Parch      Fare
0     3.0  1.0  0.271174    1.0    0.0  0.014151
1     1.0  0.0  0.472229    1.0    0.0  0.139136
2     3.0  0.0  0.321438    0.0    0.0  0.015469
3     1.0  0.0  0.434531    1.0    0.0  0.103644
4     3.0  1.0  0.434531    0.0    0.0  0.015713

In [41]:
correlation = df_train.corr()
correlation['Survived'].sort_values(ascending = False)

Survived       1.000000
Fare           0.257307
Parch          0.081629
PassengerId   -0.005007
SibSp         -0.035322
Age           -0.077221
Pclass        -0.338481
Sex           -0.543351
Name: Survived, dtype: float64

Apparently it's still the same even if you scale it!

Use the random tree forest and fit_transform the training data

In [42]:
clf = tree.DecisionTreeClassifier()